In [1]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 20871262208 (19.44 GB)
DFS Remaining: 20340154368 (18.94 GB)
DFS Used: 531107840 (506.50 MB)
DFS Used%: 2.54%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.2:9866 (project-4-neoul-bom-slow-village-makgeolli-dn-1.project-4-neoul-bom-slow-village-makgeolli_default)
Hostname: 24161900af0d
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 271425620 (258.85 MB)
Non DFS Used: 15362772908 (14.31 GB)
DFS Remaining: 1

In [2]:
# clear hdma-wi-2021.csv file
!rm -f ./hdma-wi-2021.csv

In [3]:
# download hdma-wi-2021.csv
!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv

--2023-10-26 20:21:43--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv’

hdma-wi-2021.csv    100%[===================>] 166.84M  87.7MB/s    in 1.9s    

2023-10-26 20:21:45 (87.7 MB/s) - ‘hdma-wi-2021.csv’ saved [174944099/174944099]



In [4]:
# clear csv files
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

Deleted hdfs://boss:9000/single.csv
Deleted hdfs://boss:9000/double.csv


In [5]:
# copy csv file
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp ./hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp ./hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [6]:
# clear hdma-wi-2021.csv file
!rm -f ./hdma-wi-2021.csv

In [7]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/*

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


In [8]:
import requests

In [9]:
#q3
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
r.raise_for_status()
r.json()

{'FileStatus': {'accessTime': 1698351715256,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16392,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1698351718608,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [10]:
blockSize = r.json()['FileStatus']['blockSize']
length = r.json()['FileStatus']['length']

In [11]:
#q4
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.raise_for_status()
r.json()["Location"]

'http://0732a1c305bd:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [12]:
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS")
r.raise_for_status()
blocks = r.json()['BlockLocations']['BlockLocation']  

In [13]:
#q5
id = {}
for i in range(len(blocks)):
    blocks[i] = blocks[i]['hosts'][0]
for i in blocks:
    id[i] = id.get(i, 0) + 1
id

{'0732a1c305bd': 82, '24161900af0d': 85}

In [14]:
import pyarrow as pa
import pyarrow.fs
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

2023-10-26 20:22:14,919 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
f = hdfs.open_input_file("/single.csv")

In [16]:
#q6
f.read_at(10, 0)

b'activity_y'

In [17]:
import io

In [18]:
#q7
count = 0
target = "Single Family"
with hdfs.open_input_file("/single.csv") as f:
    reader = io.TextIOWrapper(io.BufferedReader(f))
    for i, line in enumerate(reader):
        if target in line:
            count += 1
count

444874